In [1]:
from datetime import datetime
import time
import pyaudio
import wave

class Record:
    
    def __init__(self, label):
        self.chunk = 1024
        self.sample_format = pyaudio.paInt16
        self.channels = 1
        self.fs = 8000
        self.seconds = 1
        self.label = label
    
    def recording(self):
        frames = list()
        filename = f"{self.label}_{Record.random_id()}.wav"
        p = pyaudio.PyAudio()
        print('Recording')
        stream = p.open(format=self.sample_format,channels=self.channels,rate=self.fs,frames_per_buffer=self.chunk,input=True)
        for i in range(0, int(self.fs / self.chunk * self.seconds)):
            data = stream.read(self.chunk)
            frames.append(data)
    
        stream.stop_stream()
        stream.close()
        p.terminate()
        print('Finished recording')

        # Save the recorded data as a WAV file
        wf = wave.open(filename, 'wb')
        wf.setnchannels(self.channels)
        wf.setsampwidth(p.get_sample_size(self.sample_format))
        wf.setframerate(self.fs)
        wf.writeframes(b''.join(frames))
        wf.close()
      
    @staticmethod
    def random_id():
        dt = datetime.utcnow()
        stamp = time.mktime(dt.timetuple()) + dt.microsecond / 1e0
        today = time.strftime("%d%m%y")
        stamp_id = str(today) + str(stamp).split(".")[0]
        return stamp_id

In [76]:
rec = Record('noise')

In [77]:
# rec.recording()

In [83]:
import time
cnt = 0
for i in range(1,50):
    rec.recording()
    time.sleep(0.2)
    if 1000%100:
        print(cnt)
    cnt+=1

Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Finished recording
Recording
Fini

KeyboardInterrupt: 

## Data augmentation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import matplotlib.pyplot as plt
import os
import cv2
import IPython.display as ipd
from scipy.io.wavfile import write
from datetime import datetime
import time
EPS = 1e-8

In [ ]:
def random_id():
    dt = datetime.utcnow()
    stamp = time.mktime(dt.timetuple()) + dt.microsecond / 1e0
    today = time.strftime("%d%m%y")
    stamp_id = str(today) + str(stamp).split(".")[0]
    return stamp_id

In [ ]:
def data_augmentation(data, tag):
    cnt, _tag = 0, tag
    for file in data:
#         print(file)
        if cnt%100==0:
            print(cnt)
        cnt+=1
        wav, sr = librosa.load(file, sr=None)
        new_tag = None
        new_tag = f"{_tag}_time_shift"
        wav_time_shift = slice_timing(wav)
        save_audio(wav_time_shift, new_tag)
        time.sleep(0.5)
        new_tag = f"{_tag}_time_shift_speed_tune"
        wav_speed_tune = speed_tuning(wav_time_shift)
        save_audio(wav_speed_tune, new_tag)
        time.sleep(0.5)
        new_tag = f"{_tag}_actual_speed_tune"
        wav_speed_tune_actual = speed_tuning(wav)
        save_audio(wav_speed_tune_actual, new_tag)
#         print(wav.shape, wav.max(), wav.min())
#         ipd.Audio(wav, rate=sr)

In [ ]:
def get_spectrogram(wav):
    D = librosa.stft(wav, n_fft=480, hop_length=160,win_length=480, window='hamming')
    spect, phase = librosa.magphase(D)
    return spect

In [ ]:
def slice_timing(wav):
    start_ = int(np.random.uniform(-10,4800))
#     print('time shift: ',start_)
    if start_ >= 0:
        wav_time_shift = np.r_[wav[start_:], np.random.uniform(-0.001,0.001, start_)]
    else:
        wav_time_shift = np.r_[np.random.uniform(-0.001,0.001, -start_), wav[:start_]]
    #     write("example.wav", samplerate, data.astype(np.int16))
#     print(type(wav_time_shift))
    return wav_time_shift
    #     return wav_time_shift
#     ipd.Audio(wav_time_shift, rate=sr)

In [ ]:
def speed_tuning(wav):
    speed_rate = np.random.uniform(0.7,1.3)
    wav_speed_tune = cv2.resize(wav, (1, int(len(wav) * speed_rate))).squeeze()
#     print('speed rate: %.3f' % speed_rate, '(lower is faster)')
    if len(wav_speed_tune) < 16000:
        pad_len = 16000 - len(wav_speed_tune)
        wav_speed_tune = np.r_[np.random.uniform(-0.001,0.001,int(pad_len/2)),
                               wav_speed_tune,
                               np.random.uniform(-0.001,0.001,int(np.ceil(pad_len/2)))]
    else: 
        cut_len = len(wav_speed_tune) - 8000
        wav_speed_tune = wav_speed_tune[int(cut_len/2):int(cut_len/2)+8000]

    return wav_speed_tune
#     write("example.wav", samplerate, data.astype(np.int16))
#     print('wav length: ', wav_speed_tune.shape[0])
# ipd.Audio(wav_speed_tune, rate=sr)

In [ ]:
def save_audio(data, tag):
    samplerate = 8000
    write(f"augmented_{tag}_{random_id()}.wav", samplerate, data)

In [ ]:
#  samplerate = 8000; fs = 100
# t = np.linspace(0., 1., samplerate)
# amplitude = np.iinfo(np.int16).max
# data = amplitude * np.sin(2. * np.pi * fs * t)
# write("example.wav", samplerate, data.astype(np.int16))

In [ ]:
path = 'Data/Noise'

In [ ]:
data=[]

In [ ]:
data = [f'{path}/{filename}' for filename in os.listdir(path) if '.wav' in filename]
len(data), data[80]

In [ ]:
# file_path = 'Data/Human/human_1.wav'
# wav, sr = librosa.load(file_path, sr=None)
# write("example.wav", sr, wav)
# print(sr)
# ipd.Audio(wav, rate=sr)

In [ ]:
data_augmentation(data, 'noise')

In [ ]:
640*3

In [ ]:
!mv augmented_noise*.wav Data/Noise/

In [ ]:
!rm -rf Data/Noise/augmented_noise_time_shift_*.wav

In [ ]:
# !rm -rf augmented*.wav